In [1]:
import pickle
import nltk
from gensim.models import Word2Vec
import multiprocessing
import pandas as pd

In [19]:
from nltk.stem import WordNetLemmatizer

In [23]:
def word_splt(lst): 
    return list(map(lambda el:[el], lst)) 

In [2]:
!pwd

/Users/elena/Desktop/Metis/projects/5_project/Blogger_Boost/code


In [3]:
with open('../../blogger_com_data_19320_6_v2.pkl', 'rb') as picklefile:
    df = pickle.load(picklefile)

In [4]:
    with open('../../data/blogger_com_data_19320_tsne3d.pkl', 'rb') as picklefile:
        tsne = pickle.load(picklefile)

In [9]:
tsne

blogger_id        prime_topic  gender age          occupation  \
0        1000331            mundane  female  37              indUnk   
1        1000866            mundane  female  17             Student   
2        1004904  love_relationship    male  23                Arts   
3        1005076            mundane  female  25                Arts   
4        1005545           blogging    male  25         Engineering   
...          ...                ...     ...  ..                 ...   
19315     996147            mundane  female  36  Telecommunications   
19316     997488  love_relationship    male  25              indUnk   
19317     998237      teen_fun_days  female  16              indUnk   
19318     998966      teen_fun_days    male  27              indUnk   
19319     999503  love_relationship    male  25            Internet   

              name         xs         ys         zs  
0              Leo   0.281417   6.622975 -10.523910  
1            Libra -12.260028  -7.769121 -12.416444  
2        Capricorn   4.014722  -2.585350  18.365887  
3           Cancer   2.535636  -7.212558 -11.414396  
4      Sagittarius -11.344011  20.124655  -3.197232  
...            ...        ...        ...        ...  
19315          Leo  -6.202799   6.175335 -15.789851  
19316       Cancer -12.689223  -6.187814   7.715663  
19317        Virgo  -8.577352 -19.395651  -3.846501  
19318       Taurus   9.840730  -9.304145   0.778932  
19319       Cancer  -5.851444   1.147443   0.018549  

[19320 rows x 9 columns]

In [5]:
new_df = pd.merge(df, tsne,  how='left', left_on=['blogger_id'], right_on = ['blogger_id'])

In [6]:
new_df.columns

Index(['blogger_id', 'gender_x', 'age_x', 'occupation_x', 'name_x', 'post',
       'prime_topic', 'gender_y', 'age_y', 'occupation_y', 'name_y', 'xs',
       'ys', 'zs'],
      dtype='object')

In [17]:
new_df = new_df[['blogger_id', 'post', 'prime_topic']]

In [9]:
new_df_g = new_df.groupby(['prime_topic'])['post'].apply(' '.join).reset_index()

In [76]:
test = new_df_g['post'][0][:500]

In [77]:
lemmatizer=WordNetLemmatizer()
test = test.split('.')
test=word_splt(test)
test

[['        Blogdex     Blogdex  is a research project of the MIT Media Laboratory tracking the diffusion of information through the weblog community'],
 [' Ideas can have very similar properties to a disease, spreading through the population like wildfire'],
 [' The goal of Blogdex is to explore what it is about information, people, and their relationships that allows for this contagious media'],
 ['  Blogdex crawls all of the weblogs in its database every time they are updated and collects the links that have been ']]

In [79]:
tokens = [sub.split() for subl in  test for sub in subl] 
type(tokens[1])

list

In [80]:
def lemmatize_sent(sent):
    return [lemmatizer.lemmatize(word.lower()) for word in sent]

In [87]:
#lemmatizer.lemmatize('allows')

In [85]:
#tokens

In [82]:
lemmatized = []
for i in tokens:
    lemmatized.append(lemmatize_sent(i))

In [88]:
#lemmatized

In [50]:
text0_l = text0.split('.')

In [88]:
text0_l=extractDigits(text0_l)

In [76]:
#test = text0_l[0:2]

In [77]:
#test

[['        Blogdex     Blogdex  is a research project of the MIT Media Laboratory tracking the diffusion of information through the weblog community'],
 [' Ideas can have very similar properties to a disease, spreading through the population like wildfire']]

In [62]:

#newlist = [word for word in text0_l[0].split()]

In [80]:
tokens = [sub.split() for subl in  text0_l for sub in subl] 

In [82]:
embedding_dim=300

In [84]:
w_v = Word2Vec(tokens, size=embedding_dim, window=5, 
               min_count=5, negative=15, iter=10, workers=multiprocessing.cpu_count())

In [85]:
word_vectors = w_v.wv

In [92]:
type(word_vectors)

gensim.models.keyedvectors.Word2VecKeyedVectors

In [90]:
result=word_vectors.similar_by_word('love')

In [91]:
print('Most similar to "blog": ', result[:3])

Most similar to "blog":  [('hate', 0.7150373458862305), ('loved', 0.6365389823913574), ('love,', 0.6327400207519531)]


In [101]:
def lemmatize_sent(sent):
    return [lemmatizer.lemmatize(word.lower()) for word in sent]

def embedd_by_topic(ind):   
    text0 = new_df_g['post'][ind]
    
    lemmatizer=WordNetLemmatizer()
    
    text0_l = text0.split('.')
    text0_l=word_splt(text0_l)
    tokens = [sub.split() for subl in  text0_l for sub in subl] 
    
    lemmatized = []
    for i in tokens:
        lemmatized.append(lemmatize_sent(i))

    
    embedding_dim=300
    
    w_v = Word2Vec(lemmatized, size=embedding_dim, window=5, 
                   min_count=5, negative=15, iter=10, workers=multiprocessing.cpu_count())

    word_vectors = w_v.wv
    
    return word_vectors
    

In [102]:
from collections import defaultdict

embds = defaultdict()
for i in range(new_df_g['post'].shape[0]):
    embds[new_df_g['prime_topic'][i]] = embedd_by_topic(i)

In [98]:
embds

defaultdict(None,
            {'blogging': <gensim.models.keyedvectors.Word2VecKeyedVectors at 0x7fe8d1c134d0>})

In [99]:
embds['blogging'].similar_by_word('love')

[('hate', 0.8322218656539917),
 ('appreciate', 0.6973381042480469),
 ('enjoy', 0.6760803461074829),
 ('loved', 0.66237473487854),
 ('prefer', 0.6567378044128418),
 ('miss', 0.6439168453216553),
 ('agree', 0.6215038299560547),
 ('hear', 0.6065900921821594),
 ('respect', 0.5917678475379944),
 ('thank', 0.5911786556243896)]

In [103]:
with open('../../topic_embeds.pkl', 'wb') as picklefile:
    pickle.dump(embds, picklefile)

In [110]:
embds['faith'].similar_by_word('politics')

[('philosophy', 0.6688308715820312),
 ('arts', 0.6319067478179932),
 ('ethics', 0.6257492303848267),
 ('homosexuality', 0.613933265209198),
 ('Islam', 0.6097513437271118),
 ('religion', 0.6044865846633911),
 ('politics,', 0.6029326915740967),
 ('sex,', 0.5849233865737915),
 ('divorce,', 0.583962082862854),
 ('forming', 0.5829131603240967)]

In [106]:
embds.keys()

dict_keys(['blogging', 'faith', 'jargon', 'love_relationship', 'mundane', 'politics', 'teen_fun_days'])

In [107]:
embds['teen_fun_days'].similar_by_word('love')

[('luv', 0.6231053471565247),
 ('LOVE', 0.5698900818824768),
 ('Love', 0.525988757610321),
 ('loved', 0.5163581967353821),
 ('hate', 0.5134645104408264),
 ('miss', 0.5131471157073975),
 ('owe', 0.4790663421154022),
 ('love,', 0.4725012183189392),
 ('thank', 0.4432137608528137),
 ('adore', 0.4408349394798279)]

In [108]:
embds['politics'].similar_by_word('love')

[('hate', 0.5890721082687378),
 ('appreciate', 0.50534987449646),
 ('admire', 0.505074679851532),
 ('enjoy', 0.49496233463287354),
 ('miss', 0.4899393320083618),
 ('love,', 0.4881824851036072),
 ('liked', 0.4838865399360657),
 ('loved', 0.46844160556793213),
 ('prefer', 0.45760250091552734),
 ('loving', 0.4452415406703949)]

In [109]:
embds['jargon'].similar_by_word('love')

[('luv', 0.6610651016235352),
 ('loved', 0.638775110244751),
 ('love,', 0.6320081949234009),
 ('loving', 0.6222937107086182),
 ('trust', 0.6064069271087646),
 ('Love', 0.5571303963661194),
 ('respect', 0.5389784574508667),
 ('appreciate', 0.5291538834571838),
 ('treasure', 0.5153064727783203),
 ('happiness', 0.4961586594581604)]